In [1]:
import mne
import os
import numpy as np 
import pandas as pd

In [16]:
# Hjorth Parameters
def hjorth_parameters(signal):
    activity = np.var(signal)
    mobility = np.sqrt(np.var(np.diff(signal)) / activity)
    complexity = np.sqrt(np.var(np.diff(np.diff(signal))) / np.var(np.diff(signal)))
    return activity, mobility, complexity

# Bandpower Ratios
def bandpower_ratio(fft, freqs):
    delta_power = np.sum(np.abs(fft[(freqs >= 0.5) & (freqs < 4)])**2)
    theta_power = np.sum(np.abs(fft[(freqs >= 4) & (freqs < 8)])**2)
    alpha_power = np.sum(np.abs(fft[(freqs >= 8) & (freqs < 13)])**2)
    beta_power = np.sum(np.abs(fft[(freqs >= 13) & (freqs < 30)])**2)

    theta_alpha_ratio = theta_power / alpha_power if alpha_power != 0 else 0
    delta_theta_ratio = delta_power / theta_power if theta_power != 0 else 0

    return delta_power, theta_power, alpha_power, beta_power, theta_alpha_ratio, delta_theta_ratio

# Consolidated Feature Extraction Function (with Duration)
def process_edf_features(edf_file_path, output_csv_path):
    # Read file EDF using MNE
    raw = mne.io.read_raw_edf(edf_file_path, preload=True)
    first_4_channels = raw.get_data()[1::2, :]

    # Compute average signal
    sum_signal = np.sum(first_4_channels, axis=0)
    avg_signal = sum_signal / first_4_channels.shape[0]

    # Frequency domain features (FFT)
    fft = np.fft.fft(avg_signal)
    freqs = np.fft.fftfreq(len(avg_signal), d=1/raw.info['sfreq'])

    # Power in different frequency bands and bandpower ratios
    delta_power, theta_power, alpha_power, beta_power, theta_alpha_ratio, delta_theta_ratio = bandpower_ratio(fft, freqs)

    # Hjorth parameters
    activity, mobility, complexity = hjorth_parameters(avg_signal)

    # Time-domain features
    variance = np.var(avg_signal)
    power = np.mean(avg_signal ** 2)
    median = np.median(avg_signal)
    signal_range = np.max(avg_signal) - np.min(avg_signal)

    # Calculate duration of the signal
    duration = raw.times[-1]  # Duration in seconds

    # Prepare data for saving
    data = {
        'variance': [variance],
        'power': [power],
        'median': [median],
        'range': [signal_range],
        'delta_power': [delta_power],
        'theta_power': [theta_power],
        'alpha_power': [alpha_power],
        'beta_power': [beta_power],
        'theta_alpha_ratio': [theta_alpha_ratio],
        'delta_theta_ratio': [delta_theta_ratio],
        'activity': [activity],
        'mobility': [mobility],
        'complexity': [complexity],
        'duration': [duration],  # Added duration
        'status': 'epilepsy'  # Update according to patient status
    }

    # Convert to DataFrame and save
    df = pd.DataFrame(data)

    # Append or write data to CSV
    if os.path.exists(output_csv_path):
        df.to_csv(output_csv_path, mode='a', header=False, index=False)
    else:
        df.to_csv(output_csv_path, mode='w', header=True, index=False)


In [17]:
path = "process_epilepsy_20to24_1"
output_dir = "balanced2.csv"

# Loop through all EDF files in the directory
for filename in os.listdir(path):
    if filename.endswith('.edf'):
        input_dir = os.path.join(path, filename)
        
        # Process the file and extract features
        process_edf_features(input_dir, output_dir)
        
    print(f"Processed: {input_dir}")

Extracting EDF parameters from d:\FGW\EDF\v2.0.1_new\process_epilepsy_20to24_1\aaaaaanr_s001_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 303499  =      0.000 ...  1213.996 secs...
Processed: process_epilepsy_20to24_1\aaaaaanr_s001_t001.edf
Extracting EDF parameters from d:\FGW\EDF\v2.0.1_new\process_epilepsy_20to24_1\aaaaaanr_s002_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 312999  =      0.000 ...  1251.996 secs...
Processed: process_epilepsy_20to24_1\aaaaaanr_s002_t001.edf
Extracting EDF parameters from d:\FGW\EDF\v2.0.1_new\process_epilepsy_20to24_1\aaaaaanr_s003_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 331249  =      0.000 ...  1324.996 secs...
Processed: process_epilepsy_20to24_1\aaaaaanr_s003_t000.edf
Extracting EDF parameters from d:\FGW\EDF\v2.0.1_new\process_epilepsy_20to24_1\aaaaaa

In [4]:
path = "process_noepilepsy_20to24"
output_dir = "balanced.csv"

# Loop through all EDF files in the directory
for filename in os.listdir(path):
    if filename.endswith('.edf'):
        input_dir = os.path.join(path, filename)
        
        # Process the file and extract features
        process_edf_features(input_dir, output_dir)
        
    print(f"Processed: {input_dir}")

Extracting EDF parameters from d:\FGW\EDF\v2.0.1_new\process_noepilepsy_20to24\aaaaaebo_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 318749  =      0.000 ...  1274.996 secs...
Processed: process_noepilepsy_20to24\aaaaaebo_s001_t000.edf
Extracting EDF parameters from d:\FGW\EDF\v2.0.1_new\process_noepilepsy_20to24\aaaaaebo_s002_t002.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 297499  =      0.000 ...  1189.996 secs...
Processed: process_noepilepsy_20to24\aaaaaebo_s002_t002.edf
Extracting EDF parameters from d:\FGW\EDF\v2.0.1_new\process_noepilepsy_20to24\aaaaaebo_s003_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 329749  =      0.000 ...  1318.996 secs...
Processed: process_noepilepsy_20to24\aaaaaebo_s003_t001.edf
Extracting EDF parameters from d:\FGW\EDF\v2.0.1_new\process_noepilepsy_20to24\aaaaaf

In [31]:
import mne
import os

# Function to get the duration of each EDF file
def get_edf_duration(file):
    raw = mne.io.read_raw_edf(file, preload=False)  # Preload=False for faster duration check
    duration = raw.times[-1]  # The last time point gives the duration in seconds
    return duration

# Function to find and print the file with the longest duration
def find_longest_duration(path):
    longest_duration = 0
    longest_file = None
    
    # Loop through all EDF files in the directory
    for filename in os.listdir(path):
        if filename.endswith('.edf'):
            input_file = os.path.join(path, filename)
            duration = get_edf_duration(input_file)  # Get the duration of the EDF file
            
            # Check if this is the longest duration so far
            if duration > longest_duration:
                longest_duration = duration
                longest_file = input_file
    
    # Print the file with the longest duration
    if longest_file:
        print(f"The longest file is: {os.path.basename(longest_file)} with duration {longest_duration:.2f} seconds")
    else:
        print("No EDF files found.")

In [39]:
path = "process_noepilepsy_10to13"
output_csv = "test2.csv"

find_longest_duration(path)

Extracting EDF parameters from d:\FGW\EDF\v2.0.1_new\process_noepilepsy_10to13\aaaaajgj_s002_t001.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from d:\FGW\EDF\v2.0.1_new\process_noepilepsy_10to13\aaaaajgn_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from d:\FGW\EDF\v2.0.1_new\process_noepilepsy_10to13\aaaaalpi_s001_t002.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from d:\FGW\EDF\v2.0.1_new\process_noepilepsy_10to13\aaaaalpi_s002_t004.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from d:\FGW\EDF\v2.0.1_new\process_noepilepsy_10to13\aaaaalze_s001_t000.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from d:\FGW\EDF\v2.0.1_new\process

no_epilepsy 1506.00
epilepsy 2 1508.00
epilepsy 1 1506.00

11to13 787